In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date, timedelta
import time
import re


In [ ]:
def verifica_data(data: str) -> bool:
    dia = data[:2]
    mes = data[3:5]
    ano = data[-4:]

    check = False

    # meses com 31 dias
    if mes in ['01', '03', '05', '07', '08', '10', '12']:
        if int(dia) <= 31:
            check = True
    # meses com 30 dias
    elif mes in ['04', '06', '09', '11']:
        if int(dia) <= 30:
            check = True
    # fevereiro
    elif mes == '02':
        # testa se é bissexto
        if (int(ano) % 4 == 0 and int(ano) % 100 != 0) or int(ano) % 400 == 0:
            if int(dia) <= 29:
                check = True
        else:
            if int(dia) <= 28:
                check = True
    
    return check


In [ ]:
padrao_data = re.compile(r'\d{2}\/\d{2}\/\d{4}')

while True:
    # solicitar ao usuario a cidade de destino da viagem
    while True:
        destino = input('Qual a cidade de destino da viagem? ').capitalize()
        if len(destino) > 0:
            break
        else:
            display('Você precisa digitar o destino da viagem!')

    # solicitar ao ususario a data inicial de embarque
    while True:
        data_inicio = input('Qual a data inicial da pesquisa?(formato: dd/mm/aaaa) ')
        if len(re.findall(padrao_data, data_inicio)) > 0:
            if verifica_data(data_inicio):
                d_ini = date(year=int(data_inicio[-4:]), month=int(data_inicio[3:5]), day=int(data_inicio[:2]))
                if d_ini >= date.today():
                    break
                else:
                    display('A data inicial deve ser maior ou igual a data de hoje. Digite novamente.')
            else:
                display('A data digitada está no formato solicitado, porém não é uma data válida. Digite novamente.')
        else:
            display('A data digitada não está no formato esperado (dd/mm/aaaa). Digite novamente.')

    # solicitar ao usuario a data final de embarque
    while True:
        data_fim = input('Qual a data final da pesquisa?(formato: dd/mm/aaaa) ')
        if len(re.findall(padrao_data, data_fim)) > 0:
            if verifica_data(data_fim):
                d_fim = date(year=int(data_fim[-4:]), month=int(data_fim[3:5]), day=int(data_fim[:2]))
                if d_fim >= d_ini:
                    break
                else:
                    display('A data final não pode ser menor do que a data inicial. Digite novamente.')
            else:
                display('A data digitada está no formato solicitado, porém não é uma data válida. Digite novamente.')
        else:
            display('A data digitada não está no formato esperado (dd/mm/aaaa). Digite novamente.')

    # solicitar ao usuario o período da viagem em dias
    while True:
        periodo_dias = input('Digite o numero de dias da viagem? ')
        if periodo_dias.isnumeric():
            if int(periodo_dias) >= 0:
                break
            else:
                display('O período da viagem deve ser maior ou igual a 0')
        else:
            display('O período digitado não é válido. Digite novamente.')
    
    display(f'Destino da Viagem: {destino}')
    display(f'Data inicial da pesquisa: {data_inicio}')
    display(f'Data final da pesquisa: {data_fim}')
    display(f'Período da viagem: {periodo_dias}')

    confirmacao = ' '
    while confirmacao not in 'NS':
        confirmacao = input('Você confirma as informações inserida e quer continuar com a busca? [S/N] ').upper()
    if confirmacao == 'S':
        break


In [ ]:
options_chrome = Options()
options_chrome.add_argument('--window-size=1920,1080')
options_chrome.add_argument('--incognito')
options_chrome.add_argument('--headless')
# options_chrome.add_experimental_option('prefs', {'intl.accept_languages': 'pt,pt_BR'})

In [ ]:
chrome = webdriver.Chrome(options=options_chrome)
chrome.maximize_window()
chrome.implicitly_wait(5)
# chrome.get('https://www.google.com/travel/flights?tfs=CBwQARoJagcIARIDR1JVGglyBwgBEgNHUlVAAUgBcAGCAQsI____________AZgBAQ&tfu=KgIIAw')
chrome.get('https://www.google.com/travel/flights/search?tfs=CBwQAhojEgoyMDI0LTAyLTAxagcIARIDR1JVcgwIAxIIL20vMDVxdGoaIxIKMjAyNC0wMi0wOGoMCAMSCC9tLzA1cXRqcgcIARIDR1JVQAFIAXABggELCP___________wGYAQE&tfu=KgA')
actions = ActionChains(chrome)


In [ ]:
caixa_destino = chrome.find_element('xpath', '//*[@id="i21"]/div[4]/div/div/div[1]/div/div/input')
caixa_destino.clear()
caixa_destino.send_keys(destino)
time.sleep(1)
actions.send_keys(Keys.ENTER)
actions.perform()
time.sleep(1)
# chrome.find_element('xpath', '//*[@id="c57"]').click()

In [ ]:
def coleta_dados(c_day: date, p: int) -> None:
    if '--headless' in options_chrome.arguments:
        pass
        # escrever a data de embarque
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[1]/div/input').send_keys(Keys.BACK_SPACE * 30)
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[1]/div/input').send_keys(f'{str(c_day.month).zfill(2)}/{str(c_day.day).zfill(2)}/{str(c_day.year)}')
        # escrever a data de retorno
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[2]/div/input').send_keys(Keys.BACK_SPACE * 30)
        data_periodo = c_day + timedelta(days=p)
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[2]/div/input').send_keys(f'{str(data_periodo.month).zfill(2)}/{str(data_periodo.day).zfill(2)}/{str(data_periodo.year)}')
        # sair do campo da data de retorno
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[2]/div/input').send_keys(Keys.TAB)
    else:
        # clicar na data de embarque para abrir o painel de seleção de datas
        chrome.find_element('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[1]/div/input').click()
        # redefinir para limpar os campos de data
        chrome.find_element('xpath', '//div[2]/div/div[2]/div[1]/div[2]/div[2]/button/span').click()
        # escrever a data de embarque
        chrome.find_element('xpath', '//div[2]/div/div[2]/div[1]/div[1]/div[1]/div/input').send_keys(f'{str(c_day.day).zfill(2)}/{str(c_day.month).zfill(2)}/{str(c_day.year)}')
        # escrever a data de retorno
        data_periodo = c_day + timedelta(days=p)
        chrome.find_element('xpath', '//div[2]/div/div[2]/div[1]/div[1]/div[2]/div/input').send_keys(f'{str(data_periodo.day).zfill(2)}/{str(data_periodo.month).zfill(2)}/{str(data_periodo.year)}')
        # sair do campo da data de retorno para liberar o botão de confirmar
        chrome.find_element('xpath', '//div[2]/div/div[2]/div[1]/div[1]/div[2]/div/input').send_keys(Keys.TAB)

    time.sleep(1)
    # chrome.save_screenshot(f'imagens/image{c_day.day}-{data_periodo.day}.png')

    # coletar todos os preços de passagens exibidos na página
    objects = chrome.find_elements('xpath', '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[2]/div/ul/li/div/div[2]/div/div[2]/div[6]/div[1]/div[2]/span')
    print(f'Preços para viagem de {p} dias:')
    print(f'Dia de embarque {str(c_day.day).zfill(2)}/{str(c_day.month).zfill(2)}/{c_day.year}', end=' - ')
    print(f'Dia de retorno {str(data_periodo.day).zfill(2)}/{str(data_periodo.month).zfill(2)}/{data_periodo.year}', end='\n'*2)
    
    for el in objects:
        print(el.text)
    print(end='\n')
    
        

In [ ]:
print(f'Preços de Passagens para {destino}:', end='\n'*2)

current_day = d_ini
while current_day <= d_fim:
    for i in [-1, 0, 1]:
        if periodo_dias == '0' and i == -1:
            continue
        coleta_dados(current_day, int(periodo_dias) + i)
        
    # avançar para o próximo dia
    current_day += timedelta(days=1)

In [ ]:
chrome.close()